In [2]:
import pandas as pd
from evaluate import load

### Not normalized Whisper SNR (just a work code for deleted code)

In [20]:
df_not_norm = pd.read_parquet('full_testing_df.parquet.gzip')

In [21]:
df_audios = pd.read_parquet('full_dataframe_with_noises.parquet.gzip')
df_audios

,audioname,dataset,ref_orig,sampling_rate,audiopath_local,audiopath_project,noise_path,noise_class
0,mozilla-common_voice_15-23-train-2856-01818,mozilla-common_voice_15-23,"Jest także trzecia sprawa, która w czasie tej ...",16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...,.\data\UrbanSound8K\audio\fold2\156893-7-9-0.wav,jackhammer
1,pjatk-clarin_studio-15-train-0457-00001,pjatk-clarin_studio-15,dżuma wziernik przemianę księdzu krzywdzen...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...,.\data\UrbanSound8K\audio\fold1\40722-8-0-4.wav,siren
2,pjatk-clarin_mobile-15-train-0083-00007,pjatk-clarin_mobile-15,w piątek po południu była przesłuchiwana przez...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_mobile...,.\data\UrbanSound8K\audio\fold8\125678-7-0-4.wav,jackhammer
3,pwr-maleset-unk-train-0001-03097,pwr-maleset-unk,jeśli chcesz zostanę w domu,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pwr-maleset-unk-tra...,.\data\UrbanSound8K\audio\fold9\105029-7-2-16.wav,jackhammer
4,mozilla-common_voice_15-23-train-2862-00017,mozilla-common_voice_15-23,Tekst nie opiera się na żadnych podstawach nau...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...,.\data\UrbanSound8K\audio\fold10\99192-4-0-54.wav,drilling
...,...,...,...,...,...,...,...,...
4995,mailabs-corpus_librivox-19-train-2023-00011,mailabs-corpus_librivox-19,Nareszcie zniecierpliwiony kazał zamurować okn...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mailabs-corpus_libr...,.\data\VISC Dataset SON\7 (249).wav,crossover_interior
4996,pjatk-clarin_studio-15-train-0289-00016,pjatk-clarin_studio-15,dostała za ten reportaż nagrodę pulicera ...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...,.\data\VISC Dataset SON\2 (172).wav,minibus_interior
4997,mozilla-common_voice_15-23-train-2846-00448,mozilla-common_voice_15-23,Dotyczy ona zasadniczo dwóch kwestii,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...,.\data\VISC Dataset SON\5 (164).wav,jeep_interior
4998,pjatk-clarin_mobile-15-train-0035-00018,pjatk-clarin_mobile-15,każdy starał się odlecieć najbliższym samolote...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_mobile...,.\data\VISC Dataset SON\6 (638).wav,truck_interior


In [22]:
df_audios['whisper_without_noise'] = df_not_norm['whisper_no_noise']

In [23]:
df_not_norm

,audioname,dataset,ref_orig,sampling_rate,audiopath_local,audiopath_project,noise_path,noise_class,whisper_no_noise,whisper_SNR_100,whisper_SNR_10,whisper_SNR_1,audiofile_mixed,whisper_SNR_-3
0,mozilla-common_voice_15-23-train-2856-01818,mozilla-common_voice_15-23,"Jest także trzecia sprawa, która w czasie tej ...",16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...,.\data\UrbanSound8K\audio\fold2\156893-7-9-0.wav,jackhammer,"Jest także trzecia sprawa, która w czasie tej...",rycerze moi współbracia boleśni bardzośmy mał...,"Rycerze moi, współbracia boleśni, Bardzośmy m...","Rycerze moi, współbracia boleśni, Bardzośmy m...",fair-mls-20-train-0009-00038,"Rycerze moi, współbracia boleśni, Bardzośmy m..."
1,pjatk-clarin_studio-15-train-0457-00001,pjatk-clarin_studio-15,dżuma wziernik przemianę księdzu krzywdzen...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...,.\data\UrbanSound8K\audio\fold1\40722-8-0-4.wav,siren,"Dżuma, wziernik, przemianę, księdzu, krzywdze...",kantor choć guza dostał wpośród czoła gdy pos...,"Kantor, choć guza dostał w pośród czoła, gdy ...","Kantor, choć guza dostał w pośród czoła, gdy ...",fair-mls-20-train-0009-00044,"Kantor, choć guza dostał w pośród czoła, gdy ..."
2,pjatk-clarin_mobile-15-train-0083-00007,pjatk-clarin_mobile-15,w piątek po południu była przesłuchiwana przez...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_mobile...,.\data\UrbanSound8K\audio\fold8\125678-7-0-4.wav,jackhammer,w piątek po południu była przesłuchiwana prze...,przeszłe przypadki gdy dobrze pamięta zmyśla ...,"Przeszłe przypadki, gdy dobrze pamięta Zmyśla...","Przeszły przypadki, gdy dobrze pamięta, Myśla...",fair-mls-20-train-0009-00067,"Przeszło przypadki, gdy dobrze pamięta, myśli..."
3,pwr-maleset-unk-train-0001-03097,pwr-maleset-unk,jeśli chcesz zostanę w domu,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pwr-maleset-unk-tra...,.\data\UrbanSound8K\audio\fold9\105029-7-2-16.wav,jackhammer,Jeśli chcesz zostanę w domu.,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,fair-mls-20-train-0009-00079,"Nie przykrzy własnym hołdownikom, Ani swemu m..."
4,mozilla-common_voice_15-23-train-2862-00017,mozilla-common_voice_15-23,Tekst nie opiera się na żadnych podstawach nau...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...,.\data\UrbanSound8K\audio\fold10\99192-4-0-54.wav,drilling,Tekst nie opiera się na żadnych podstawach na...,pięknież to przecie patrzeć na świat z góry w...,"Pięknież to przecie patrzeć na świat z góry, ...","Pięknież to przecie patrzyć na świat z góry, ...",fair-mls-20-train-0009-00080,"Piękny, że to przecie patrzyć na świat z góry..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,mailabs-corpus_librivox-19-train-2023-00011,mailabs-corpus_librivox-19,Nareszcie zniecierpliwiony kazał zamurować okn...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mailabs-corpus_libr...,.\data\VISC Dataset SON\7 (249).wav,crossover_interior,Nareszcie zniecierpliwiony kazał zamurować ok...,Duże litery.,Duże litery.,W tłu że litery.,pwr-viu-unk-train-0001-04231,Służę literę.
4996,pjatk-clarin_studio-15-train-0289-00016,pjatk-clarin_studio-15,dostała za ten reportaż nagrodę pulicera ...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...,.\data\VISC Dataset SON\2 (172).wav,minibus_interior,"Dostała zatem reportaż nagrodę Pulitzera, ale...",Duże litery.,Duże litery.,Duże litery.,pwr-viu-unk-train-0001-04241,Duże litery.
4997,mozilla-common_voice_15-23-train-2846-00448,mozilla-common_voice_15-23,Dotyczy ona zasadniczo dwóch kwestii,16000,C:\Users\Eryk\.cache\huggingface\datasets\down

In [24]:
import pandas as pd

# Assuming your dataframe is named df
# Replace df with the actual name of your dataframe if different

# Create a new dataframe for 'audioname' and 'ref_orig'
df_audioname = df_audios[['audioname', 'ref_orig','whisper_without_noise','noise_class']].drop_duplicates()

# Create a new dataframe for 'audiofile_mixed' and 'SNR_100'
df_audiofile = df_not_norm[['audiofile_mixed', 'whisper_SNR_100','whisper_SNR_10','whisper_SNR_1','whisper_SNR_-3']].drop_duplicates()

# Merge the two dataframes on 'audioname' and 'audiofile_mixed'
result_df = pd.merge(df_audioname, df_audiofile, how='outer', left_on='audioname', right_on='audiofile_mixed')

# Drop the redundant column and rename columns
result_df = result_df.drop(columns=['audiofile_mixed']).rename(columns={'ref_orig': 'sentences'})

# Display the result_df
print(result_df)

                         audioname  \
0     fair-mls-20-train-0009-00038   
1     fair-mls-20-train-0009-00044   
2     fair-mls-20-train-0009-00067   
3     fair-mls-20-train-0009-00079   
4     fair-mls-20-train-0009-00080   
...                            ...   
4995  pwr-viu-unk-train-0001-04231   
4996  pwr-viu-unk-train-0001-04241   
4997  pwr-viu-unk-train-0001-04251   
4998  pwr-viu-unk-train-0001-04261   
4999  pwr-viu-unk-train-0001-04267   

                                              sentences  \
0     rycerze moi współ bracia boleśni bardzośmy mał...   
1     kantor choć guza dostał wpośród czoła gdy post...   
2     przeszłe przypadki gdy dobrze pamięta zmyśla g...   
3     nie przykrzy własnym hołdownikom ani swemu mon...   
4     nieskończenie pięknieżto przecie patrzać na św...   
...                                                 ...   
4995                                        duże litery   
4996                                        duże litery   
4997        

In [25]:
result_df

,audioname,sentences,whisper_without_noise,noise_class,whisper_SNR_100,whisper_SNR_10,whisper_SNR_1,whisper_SNR_-3
0,fair-mls-20-train-0009-00038,rycerze moi współ bracia boleśni bardzośmy mał...,rycerze moi współbracia boleśni bardzośmy mał...,minibus_interior,rycerze moi współbracia boleśni bardzośmy mał...,"Rycerze moi, współbracia boleśni, Bardzośmy m...","Rycerze moi, współbracia boleśni, Bardzośmy m...","Rycerze moi, współbracia boleśni, Bardzośmy m..."
1,fair-mls-20-train-0009-00044,kantor choć guza dostał wpośród czoła gdy post...,kantor choć guza dostał wpośród czoła gdy pos...,siren,kantor choć guza dostał wpośród czoła gdy pos...,"Kantor, choć guza dostał w pośród czoła, gdy ...","Kantor, choć guza dostał w pośród czoła, gdy ...","Kantor, choć guza dostał w pośród czoła, gdy ..."
2,fair-mls-20-train-0009-00067,przeszłe przypadki gdy dobrze pamięta zmyśla g...,przeszłe przypadki gdy dobrze pamięta zmyśla ...,jackhammer,przeszłe przypadki gdy dobrze pamięta zmyśla ...,"Przeszłe przypadki, gdy dobrze pamięta Zmyśla...","Przeszły przypadki, gdy dobrze pamięta, Myśla...","Przeszło przypadki, gdy dobrze pamięta, myśli..."
3,fair-mls-20-train-0009-00079,nie przykrzy własnym hołdownikom ani swemu mon...,nie przykrzy własnym hołdownikom ani swemu mo...,crossover_interior,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,"Nie przykrzy własnym hołdownikom, Ani swemu m..."
4,fair-mls-20-train-0009-00080,nieskończenie pięknieżto przecie patrzać na św...,pięknież to przecie patrzeć na świat z góry w...,sports_car_interior,pięknież to przecie patrzeć na świat z góry w...,"Pięknież to przecie patrzeć na świat z góry, ...","Pięknież to przecie patrzyć na świat z góry, ...","Piękny, że to przecie patrzyć na świat z góry..."
...,...,...,...,...,...,...,...,...
4995,pwr-viu-unk-train-0001-04231,duże litery,Duże litery.,siren,Duże litery.,Duże litery.,W tłu że litery.,Służę literę.
4996,pwr-viu-unk-train-0001-04241,duże litery,Duże litery.,siren,Duże litery.,Duże litery.,Duże litery.,Duże litery.
4997,pwr-viu-unk-train-0001-04251,małe litery,Małe litery.,car_horn,Małe litery.,Małe litery.,mała litarz,Mała literka.
4998,pwr-viu-unk-train-0001-04261,małe litery,Małe litele.,drilling,Małe litele.,Mała Litera,Mało mi wcale.,Mało mi tego.


In [26]:
#saving all work to parquet
result_df.to_parquet('./data/parquets/not_normalized.parquet.gzip', compression = 'gzip')

### Whisper not normalized

In [27]:
df_not_norm = pd.read_parquet('./data/parquets/not_normalized.parquet.gzip')

In [28]:
df_not_norm

,audioname,sentences,whisper_without_noise,noise_class,whisper_SNR_100,whisper_SNR_10,whisper_SNR_1,whisper_SNR_-3
0,fair-mls-20-train-0009-00038,rycerze moi współ bracia boleśni bardzośmy mał...,rycerze moi współbracia boleśni bardzośmy mał...,minibus_interior,rycerze moi współbracia boleśni bardzośmy mał...,"Rycerze moi, współbracia boleśni, Bardzośmy m...","Rycerze moi, współbracia boleśni, Bardzośmy m...","Rycerze moi, współbracia boleśni, Bardzośmy m..."
1,fair-mls-20-train-0009-00044,kantor choć guza dostał wpośród czoła gdy post...,kantor choć guza dostał wpośród czoła gdy pos...,siren,kantor choć guza dostał wpośród czoła gdy pos...,"Kantor, choć guza dostał w pośród czoła, gdy ...","Kantor, choć guza dostał w pośród czoła, gdy ...","Kantor, choć guza dostał w pośród czoła, gdy ..."
2,fair-mls-20-train-0009-00067,przeszłe przypadki gdy dobrze pamięta zmyśla g...,przeszłe przypadki gdy dobrze pamięta zmyśla ...,jackhammer,przeszłe przypadki gdy dobrze pamięta zmyśla ...,"Przeszłe przypadki, gdy dobrze pamięta Zmyśla...","Przeszły przypadki, gdy dobrze pamięta, Myśla...","Przeszło przypadki, gdy dobrze pamięta, myśli..."
3,fair-mls-20-train-0009-00079,nie przykrzy własnym hołdownikom ani swemu mon...,nie przykrzy własnym hołdownikom ani swemu mo...,crossover_interior,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,nie przykrzy własnym hołdownikom ani swemu mo...,"Nie przykrzy własnym hołdownikom, Ani swemu m..."
4,fair-mls-20-train-0009-00080,nieskończenie pięknieżto przecie patrzać na św...,pięknież to przecie patrzeć na świat z góry w...,sports_car_interior,pięknież to przecie patrzeć na świat z góry w...,"Pięknież to przecie patrzeć na świat z góry, ...","Pięknież to przecie patrzyć na świat z góry, ...","Piękny, że to przecie patrzyć na świat z góry..."
...,...,...,...,...,...,...,...,...
4995,pwr-viu-unk-train-0001-04231,duże litery,Duże litery.,siren,Duże litery.,Duże litery.,W tłu że litery.,Służę literę.
4996,pwr-viu-unk-train-0001-04241,duże litery,Duże litery.,siren,Duże litery.,Duże litery.,Duże litery.,Duże litery.
4997,pwr-viu-unk-train-0001-04251,małe litery,Małe litery.,car_horn,Małe litery.,Małe litery.,mała litarz,Mała literka.
4998,pwr-viu-unk-train-0001-04261,małe litery,Małe litele.,drilling,Małe litele.,Mała Litera,Mało mi wcale.,Mało mi tego.


In [29]:
snr_list = ['whisper_without_noise', 'whisper_SNR_100',	'whisper_SNR_10','whisper_SNR_1','whisper_SNR_-3']
for snr_column in snr_list:
    wer = load("wer")
    snr_value_string = '_'.join(snr_column.split('_')[2:3])
    wer_score = wer.compute(predictions=df_not_norm[snr_column], references=df_not_norm['sentences'])
    print( f"Whisper model WER score for {snr_value_string}: {round(wer_score * 100,4)}")

Whisper model WER score for noise: 18.2771
Whisper model WER score for 100: 19.3506
Whisper model WER score for 10: 27.7929
Whisper model WER score for 1: 36.7453
Whisper model WER score for -3: 47.3158


### Whisper SNR evaluation

In [ ]:
df_audio = pd.read_parquet('./data/parquets/Whisper_SNR_WER.gzip')

In [ ]:
print(df_audio.columns.to_list())

In [8]:
snr_list = ['WER_audio_SNR_100_path', 'WER_audio_SNR_50_path', 'WER_audio_SNR_25_path', 'WER_audio_SNR_10_path', 'WER_audio_SNR_5_path', 'WER_audio_SNR_0.1_path', 'WER_audio_SNR_-1_path', 'WER_audio_SNR_-3_path', 'WER_audio_SNR_-10_path']
for snr_column in snr_list:
    wer = load("wer")
    snr_value_string = '_'.join(snr_column.split('_')[2:4])
    wer_score = wer.compute(predictions=df_audio[snr_column], references=df_audio['ref_orig'])
    print( f"Whisper model WER score for {snr_value_string}: {round(wer_score * 100,4)}")

Whisper model WER score for SNR_100: 19.3399
Whisper model WER score for SNR_50: 19.584
Whisper model WER score for SNR_25: 22.4602
Whisper model WER score for SNR_10: 28.2408
Whisper model WER score for SNR_5: 32.7553
Whisper model WER score for SNR_0.1: 42.6143
Whisper model WER score for SNR_-1: 46.2866
Whisper model WER score for SNR_-3: 54.5155
Whisper model WER score for SNR_-10: 98.1438


In [7]:
df_audio

,audioname,dataset,ref_orig,sampling_rate,audiopath_bigos,audiopath_local,noise_path,noise_class,normalised_audio_path,normalised_noise_path,...,audio_SNR_-10_path,WER_audio_SNR_100_path,WER_audio_SNR_50_path,WER_audio_SNR_25_path,WER_audio_SNR_10_path,WER_audio_SNR_5_path,WER_audio_SNR_0.1_path,WER_audio_SNR_-1_path,WER_audio_SNR_-3_path,WER_audio_SNR_-10_path
0,fair-mls-20-train-0009-04739,fair-mls-20,tam nocne włóczęgi wołano z dachów jeżeli nie ...,16000,fair-mls-20-train-0009-04739.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold9\79089-0-0-106.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,hej wy tam nocnemarki wołano z dachów jeżeli ...,hej wy tam nocnemarki wołano z dachów jeżeli ...,"Hej, wy tam, nocne marki! wołano z dachów. Je...","— Hej, wy tam, nocne marki! — wołano z dachów...","Hej, wy tam, nocne marki! Wołamy z dachu. Jeż...","Hej, byj tam, nocne marki! Chyła mu z dachu. ...","Hej, witam, Nocne Marki! Chyba nie zdarzył. J...","Hej, witam, Marcin. Nie mam uwagi. Jeżeli nie...",Dzięki za oglądanie!
1,pjatk-clarin_studio-15-train-0488-00003,pjatk-clarin_studio-15,w pracy studenci chcieliby przede wszystk...,16000,pjatk-clarin_studio-15-train-0488-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold10\167750-4-1-0.wav,drilling,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\pjatk-clarin_s...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,Pracy studenci chcieliby przede wszystkim ods...,Pracy studenci chcieliby przede wszystkim odb...,Pracy studentów chcieliby przede wszystkim od...,Przede wszystkim chcieliby studenci realizowa...,Dziękuję.
2,fair-mls-20-train-0009-05501,fair-mls-20,co to znaczy sam siebie zapytywał faraon czy g...,16000,fair-mls-20-train-0009-05501.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold3\165039-7-5-0.wav,jackhammer,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,co to znaczy sam siebie zapytywał faraon czy ...,co to znaczy sam siebie zapytywał faraon czy ...,co to znaczy sam siebie zapytywał faraon czy ...,Co to znaczy? sam siebie zapytywał Faraon. Cz...,Co to znaczy? sam siebie zapytał Faraon. Czy ...,Co to znaczy? Sam siebie zapytał paraon. Czy ...,To co znasz? Tam siebie zapytował paraon. Prz...,"Otóż, pan się wychowywał parami przy koło lud...",Dziękuje za oglądanie.
3,fair-mls-20-train-0021-01519,fair-mls-20,tylko na piaszczystem wybrzeżu lub na łąkach b...,16000,fair-mls-20-train-0021-01519.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold1\57320-0-0-22.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,tylko na piaszczystem wybrzeżu lub na łąkach ...,tylko na piaszczystem wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...
4,pjatk-clarin_studio-15-train-0335-00001,pjatk-clarin_studio-15,zaokrągla uziemienie księdzu liźnięcie rol...,16000,pjatk-clarin_studio-15-train-0335-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold4\146709-0-0-20.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./

In [9]:
df_wav2wec = pd.read_parquet('./data/parquets/wav2wec_wer.parquet.gzip')
df_wav2wec

,audio_name,sentences,Wav2wec_SNR_100,Wav2wec_SNR_50,Wav2wec_SNR_25,Wav2wec_SNR_10
0,fair-mls-20-train-0009-04739.wav,tam nocne włóczęgi wołano z dachów jeżeli nie ...,hej wytam nocne marki - wołano z dachów - jeże...,hej wytam nocne marki - wołano z dachów - jeże...,hej wytam nocne marki wołano d dachów - jeżel...,hej wytam nocne marki wołano z dachób jeżeli...
1,pjatk-clarin_studio-15-train-0488-00003.wav,w pracy studenci chcieliby przede wszystk...,w pracy studenci chcieliby przede wszystkim od...,w pracy studenci chcieliby przede wszystkim od...,pracy studenci chcieliby przede wszystkim odcz...,traty studenci ścielidy przede wszystkim odczu...
2,fair-mls-20-train-0009-05501.wav,co to znaczy sam siebie zapytywał faraon czy g...,co to znaczy sam siebie zapytywał faraon czy...,co to znaczy sam siebie zapytywał faraon czy...,co to znaczy sam siebie zapytywał faraon czy ...,co co znaczy sam siebie zapytywał taraon czy g...
3,fair-mls-20-train-0021-01519.wav,tylko na piaszczystem wybrzeżu lub na łąkach b...,tylko na piaszczystem wybrzeżu lub na łąkach b...,tylko na piaszczystem wybrzeżu lub na łąkach b...,tylko na piaszczystem wybrzeżu lub na łąkach b...,tylko na piaszczystem wybrzeżu lub na łąkach b...
4,pjatk-clarin_studio-15-train-0335-00001.wav,zaokrągla uziemienie księdzu liźnięcie rol...,zaokrągle uziemienie księdzu liźnięcie role lo...,zaokrągle uziemienie księdzu liźnięcie role lo...,zaokrągle uziemienie księdzu liźnięcie role lo...,za okrągle uziemienie księdzu liźnięcie role l...
...,...,...,...,...,...,...
2495,fair-mls-20-train-0009-06517.wav,kazał zrobić spis wszystkich mężczyzn w państw...,kazał zrobić spis wszystkich mężczyznón z pańs...,kazał zrobić spis wszystkich mężczyznón z pańs...,kazał zrobić spis wszystkich mężczyznón z pańs...,kazał zrobić pis wszystkich mężczyznów z państ...
2496,mozilla-common_voice_15-23-train-2851-00218.wav,"W odniesieniu do Lizbony, uczyniliśmy także po...",po dniesieniu do lizbony uczyniliśmy także pos...,po dniesieniu do lizbony uczyniliśmy także pos...,po dniesieniu do lizbony uczyniliśmy także pos...,podniesieniu do lizbony uczyniliśmy także post...
2497,mozilla-common_voice_15-23-train-2856-01361.wav,"Jej budżet to budżet, który wspiera inwestycje",jej pudszed to budrzet który wspiera inwestycję,jej pudszet to budrzet który wspiera inwestycję,jej budrzed to budrzet który wspiera inwestycję,jej budrzet to budrzet który wzpiera inbescycję
2498,fair-mls-20-train-0009-03165.wav,upłynęło już kilka godzin po zachodzie słońca ...,upłynęło już kilka godzin po zachodzie słońca ...,upłynęło już kilka godzin po zachodzie słońca ...,upłynęło już kilka godzin po zachodzie słońca ...,upłynęło już kilka godzin po zachodzie słońca ...


In [10]:
df = df_wav2wec
snr_list = ['Wav2wec_SNR_100', 'Wav2wec_SNR_50','Wav2wec_SNR_25', 'Wav2wec_SNR_10']
for snr_column in snr_list:
    wer = load("wer")
    wer_score = wer.compute(predictions=df[snr_column], references=df['sentences'])
    print( f"Whisper model WER score for {snr_column}: {round(wer_score * 100,4)}")

Whisper model WER score for Wav2wec_SNR_100: 23.4883
Whisper model WER score for Wav2wec_SNR_50: 23.5924
Whisper model WER score for Wav2wec_SNR_25: 25.0105
Whisper model WER score for Wav2wec_SNR_10: 38.5499
